In [7]:
import json
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from re import sub


In [17]:
import json
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from re import sub

def getLinksFromAllPages():
    pageBaseURL = 'https://www.myarkansaslottery.com/games/instant?amount=All&page='
    allLinks = []
    pageNumber = 0
    while True:
        nextURL = pageBaseURL+str(pageNumber)
        response = requests.get(nextURL)
        soup = BeautifulSoup(response.content)
        links = getPageLinks(soup)
        if len(links) == 0:
            break
        allLinks.extend(links)
        pageNumber += 1        
    return allLinks

def getPageLinks(soup):
    gameBaseURL = 'https://www.myarkansaslottery.com'
    pageContent = soup.find("div",{"class":"view-content"})
    if pageContent is None:
        return []
    gameMetaData = pageContent.findAll("h2")
    links = [gameBaseURL+game.find("a")["href"] for game in gameMetaData]
    return links

def stringToFloat(s):
     return float(sub(r'[^\d.]', '', s))   
    
def getGameName(gameTag):
    return gameTag.h1.text

def getGameCost(gameTag):
    div = gameTag.find("div",{"class":"field field-name-field-ticket-price field-type-text field-label-above layout-3col__col-x"})
    asString = div.find("div",{"class":"field-item even"}).text
    asFloat = stringToFloat(asString)
    return asFloat

def getPrizeOdds(gameTag):
    div = gameTag.find("div",{"class":"field field-name-field-game-odds field-type-text field-label-above layout-3col__col-x"})
    asString = div.find("div",{"class":"field-item even"}).text.split(' ')[-1]
    asFloat = stringToFloat(asString)
    return asFloat
    
def getTotalAmountRemaining(gameTag):
    css_selector = 'div.layout-center:nth-child(7) > div:nth-child(1)'
    div = gameTag.select(css_selector)
    _,_,_,asString = list(gameTag.select('div.layout-center:nth-child(7) > div:nth-child(1)'))[0].text.split(" ")
    asFloat = stringToFloat(asString)
    return asFloat

def getGameTableData(gameTag):
    rows = gameTag.html.body.table.tbody.findAll("tr")
    table = []
    for row in rows:
        rowData = {}
        rowData['Tier'] = row.find("td",{"data-cell-title":"Tier Prize Description:"}).text
        rowData['TotalPrizes'] = stringToFloat(row.find("td",{"data-cell-title":"Total Prizes:"}).text)
        rowData['RemainingPrizes']= stringToFloat(row.find("td",{"data-cell-title":"Total Prizes:"}).text)
        rowData['TotalValue'] = stringToFloat(row.find("td",{"data-cell-title":"Total Prize Amount:"}).text)
        rowData['RemainingValue'] = stringToFloat(row.find("td",{"data-cell-title":"Estimated Prize Amount Remaining:"}).text)
        table.append(rowData)
    return table

def scrapeGamePage(gameTag):
    gameData = {}
    gameData['Name'] = getGameName(gameTag)
    gameData['Cost'] = getGameCost(gameTag)
    gameData['Odds'] = getPrizeOdds(gameTag)
    gameData['Data'] = getGameTableData(gameTag)
    return gameData

def scrapeAllGames():
    links = getLinksFromAllPages()
    gamesData = []
    for link in links:
        response = requests.get(link)
        soup = BeautifulSoup(response.content)
        gamesData.append(scrapeGamePage(soup))
    return gamesData

gamesData = scrapeAllGames()
gamesData

with open (f"data/arkansas/{today}_ARscratchers.json", "w") as file:
    json.dump(gamesData, file)

In [18]:
gamesData = scrapeAllGames()
gamesData

[{'Name': 'Multiplier Mayhem',
  'Cost': 10.0,
  'Odds': 2.99,
  'Data': [{'Tier': '$300,000',
    'TotalPrizes': 3.0,
    'RemainingPrizes': 3.0,
    'TotalValue': 900000.0,
    'RemainingValue': 900000.0},
   {'Tier': '$10,000',
    'TotalPrizes': 2.0,
    'RemainingPrizes': 2.0,
    'TotalValue': 20000.0,
    'RemainingValue': 20000.0},
   {'Tier': '$5,000',
    'TotalPrizes': 2.0,
    'RemainingPrizes': 2.0,
    'TotalValue': 10000.0,
    'RemainingValue': 10000.0},
   {'Tier': '$1,000',
    'TotalPrizes': 70.0,
    'RemainingPrizes': 70.0,
    'TotalValue': 70000.0,
    'RemainingValue': 70000.0},
   {'Tier': '$500',
    'TotalPrizes': 327.0,
    'RemainingPrizes': 327.0,
    'TotalValue': 163500.0,
    'RemainingValue': 162000.0},
   {'Tier': '$200',
    'TotalPrizes': 525.0,
    'RemainingPrizes': 525.0,
    'TotalValue': 105000.0,
    'RemainingValue': 104800.0},
   {'Tier': '$100',
    'TotalPrizes': 27156.0,
    'RemainingPrizes': 27156.0,
    'TotalValue': 2715600.0,
    'Re